[805. Split Array With Same Average](https://leetcode.com/problems/split-array-with-same-average/)

设`B`长度为`i`, `C`长度为`j`, 相同的平均值为`n`, 则`A`的总和可被表示为`(i + j) * n`(`i,j >= 1`, `i + j = len(A)`). 即`A`的平均值也为`n`.

所以目标从将`A`拆分为`B`和`C`, 变为从`A`找一个平均值等于`A`的平均值的子序列, 那么这个子序列可以看做是`B`, 剩余的元素组成`C`, 我们就找到了问题的解.

此处可以共享的计算量是子序列的长度以及和, 所以可以用记录之前所有可能的子序列长度&和 + one-pass的方式计算.

In [39]:
class Solution:
    def splitArraySameAverage(self, A) -> bool:
        length = len(A)
        avg = sum(A) / length
        
        # dp[i]表示长度为i的子序列可以达到的和
        from collections import defaultdict
        dp = [set() for _ in range(length + 1)]
        dp[0] = {0}
        
        
        for a in A:
            # print(a)
            # 长度为i和为dp[i][j]的序列可以增加a, 变为长度为i+1和为dp[i][j] + a的序列
            for i in range(length - 1, -1, -1):
                for j in dp[i]:
                    # print(i + 1, j + a)
                    dp[i + 1].add(j + a)
        
        # print(dp)

        # 检查每个子序列长度, 看它的平均值是否为avg
        for each_len in range(1, length):
            for total in dp[each_len]:
                if total / each_len == avg:
                    # print(each_len, total, avg)
                    return True
    
        return False

In [40]:
Solution().splitArraySameAverage([17, 3, 7, 12, 1])

False

In [30]:
sum([17, 3, 7, 12, 1]) - 32

8

学习Stefan大神的解法, 有以下值得学习的点

- DP部分可以用位操作加速, `P[n]`的第`i`位表示可以组成长度为`n`, 总和为`a`的子序列
- 不需要对每一个和判断是否满足条件, 而是看满足条件的数是不是可以达到的和

In [43]:
class Solution:
    def splitArraySameAverage(self, A) -> bool:
        N, S, P = len(A), sum(A), [1]
        for a in A:
            # P[0]永远是1, 虽然实际上是0, 但是为了后面的移位操作初始化成了1, 反正最后判断的时候不需要P[0]
            # 先取出P, P[1:]+[0],相当于相邻元素进行操作的one-pass
            # p为长度为i的子序列可以达到的和, 加上a后每个和都可以左移a位, 此处为"新值"
            # q为长度为i+1的子序列可以达到的和, 此处相当于"旧值"
            # "新值" "旧值"取OR, 得到结果
            P[1:] = [(p << a) | q for p, q in zip(P, P[1:] + [0])]
            
        # for n in range(1, N): 逐长度判断
        #
        # P[n] & (1 << (S * n // N)):
        # 长度为n时, 要达到和A相同平均值, 需要的子序列和为(S // N) * n, 此处变换位置防止运算错误
        # 通过&(1<<k)的方式获取低位起第k位
        #
        # S * n % N == 0:
        # 上面提到子序列和为s = (S // N) * n
        # 合法的情况当且仅当(S * n) % N == 0
        
        return any(S * n % N == 0 and P[n] & (1 << (S * n // N))
                   for n in range(1, N))